In [ ]:
import jax

jax.config.update("jax_enable_x64", True)

import jax.numpy as jnp
import matplotlib.pyplot as plt

import optimal_control.controls as controls
from optimal_control.environments.examples.active_fibrosis import (
    FibrosisEnvironment,
    FibrosisState,
)

In [ ]:
environment: FibrosisEnvironment = FibrosisEnvironment()
#influx_sequence = environment._sample_influx_sequence_twostate(t0=0.0, t1=1000.0, dt=1.0, p01=0.01, p10=0.25, key=jax.random.PRNGKey(1))
influx_sequence = environment._sample_influx_sequence_random(t0=0.0, t1=1000.0, dt=4.0, p1=0.04, key=jax.random.PRNGKey(1))

plt.figure()
plt.plot(influx_sequence)
plt.show()

In [ ]:
def copy_data(_, data):
    return data

In [ ]:
key = jax.random.PRNGKey(1)
state = environment.init()
ys = environment.integrate(controls.LambdaControl(copy_data, jnp.full((2,), 0.01)), state, key)

In [ ]:
ys.stats

In [ ]:
plt.figure()
plt.yscale("log")
plt.plot(ys.ys)
plt.show()

In [ ]:
ys